In [1]:
!pip install flask flask-mysqldb pandas plotly openpyxl




[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install xlrd==2.0.1


   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ------------ --------------------------- 30.7/96.5 kB 330.3 kB/s eta 0:00:01
   ------------------------- -------------- 61.4/96.5 kB 409.6 kB/s eta 0:00:01
   ---------------------------------------- 96.5/96.5 kB 504.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## LOADING THE DATA

In [22]:
import pandas as pd

In [23]:
df_5308 = pd.read_excel('5308.xls', sheet_name=None)
df_5329 = pd.read_excel('5329.xls', sheet_name=None)

In [24]:
current_5308 = df_5308['Detail_67_3_5'].iloc[:, 5]
voltage_5308 = df_5308['Detail_67_3_5'].iloc[:, 6]
capacity_5308 = df_5308['Detail_67_3_5'].iloc[:, 7]
temperature_5308 = df_5308['DetailTemp_67_3_5'].iloc[:, 4]
time_5308 = df_5308['Detail_67_3_5'].iloc[:, 10]

current_5329 = df_5329['Detail_67_3_1'].iloc[:, 5]
voltage_5329 = df_5329['Detail_67_3_1'].iloc[:, 6]
capacity_5329 = df_5329['Detail_67_3_1'].iloc[:, 7]
temperature_5329 = df_5329['DetailTemp_67_3_1'].iloc[:, 4]
time_5329 = df_5329['Detail_67_3_1'].iloc[:, 10]


In [25]:
soh_5308 = (2992.02 / 3000) * 100
soh_5329 = (2822.56 / 3000) * 100

print(soh_5308)
print(soh_5329)


99.734
94.08533333333334


## Connection with MySQL

In [30]:
import pandas as pd


In [ ]:
import MySQLdb

db = MySQLdb.connect("127.0.0.1", "root", "1000", "Numan")
cursor = db.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS Numan;")
cursor.execute("USE Numan;")
cursor.execute("CREATE TABLE IF NOT EXISTS cell_data (id INT AUTO_INCREMENT PRIMARY KEY, cell_id INT, current FLOAT, voltage FLOAT, capacity FLOAT, temperature FLOAT, time FLOAT);")
cursor.execute("CREATE TABLE IF NOT EXISTS soh_data (cell_id INT PRIMARY KEY, discharge_capacity FLOAT, nominal_capacity FLOAT, soh FLOAT);")

cursor.execute("INSERT INTO soh_data (cell_id, discharge_capacity, nominal_capacity, soh) VALUES (5308, 2992.02, 3000, %s) ON DUPLICATE KEY UPDATE soh=%s", (soh_5308, soh_5308))
cursor.execute("INSERT INTO soh_data (cell_id, discharge_capacity, nominal_capacity, soh) VALUES (5329, 2822.56, 3000, %s) ON DUPLICATE KEY UPDATE soh=%s", (soh_5329, soh_5329))

for i in range(len(current_5308)):
    cursor.execute("INSERT INTO cell_data (cell_id, current, voltage, capacity, temperature, time) VALUES (5308, %s, %s, %s, %s, %s)",
                   (current_5308[i], voltage_5308[i], capacity_5308[i], temperature_5308[i], time_5308[i]))

for i in range(len(current_5329)):
    cursor.execute("INSERT INTO cell_data (cell_id, current, voltage, capacity, temperature, time) VALUES (5329, %s, %s, %s, %s, %s)",
                   (current_5329[i], voltage_5329[i], capacity_5329[i], temperature_5329[i], time_5329[i]))

db.commit()
db.close()


In [ ]:
import pandas as pd
import MySQLdb
import re


def sanitize_column_name(col_name):
    col_name = col_name.strip()
    col_name = re.sub(r'[^a-zA-Z0-9_]', '_', col_name)
    return col_name

# Function to create MySQL table from DataFrame
def create_table_from_df(cursor, df, table_name):
    cols = []
    for col_name, dtype in zip(df.columns, df.dtypes):
        col_name = sanitize_column_name(col_name)
        if "int" in str(dtype):
            col_type = "INT"
        elif "float" in str(dtype):
            col_type = "FLOAT"
        else:
            col_type = "TEXT"
        cols.append(f"{col_name} {col_type}")
    cols_str = ", ".join(cols)
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({cols_str});")

# Function to insert data into MySQL table from DataFrame
def insert_data_from_df(cursor, df, table_name):
    cols = ", ".join([sanitize_column_name(col) for col in df.columns])
    placeholders = ", ".join(["%s"] * len(df.columns))
    for row in df.replace({pd.NA: None}).itertuples(index=False, name=None):
        cursor.execute(f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders});", row)

# Connect to MySQL database
db = MySQLdb.connect("127.0.0.1", "root", "1000", "Numan")
cursor = db.cursor()

# Drop all existing tables
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'Numan';")
tables = cursor.fetchall()
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table[0]};")

# Load Excel files
files = ['5308.xls', '5329.xls']
for file in files:
    xls = pd.ExcelFile(file)
    cell_id = file.split('.')[0]

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file, sheet_name=sheet_name)
        
        # Sanitize column names in the DataFrame
        df.columns = [sanitize_column_name(col) for col in df.columns]

        # Create table name using cell_id and sheet_name
        table_name = f"{cell_id}_{sanitize_column_name(sheet_name)}"

        # Create table
        create_table_from_df(cursor, df, table_name)
        # Insert data into table
        insert_data_from_df(cursor, df, table_name)

# Commit and close the database connection
db.commit()
db.close()

print("Data has been successfully read and tables have been created in the MySQL database.")
